## Decision Tree (classification)

### Gini impurity

 * with $c$ classes:
$$I_G(p)=1 - \sum_{j=1}^c p_j^2$$

In [7]:
import math
from collections import Counter

In [ ]:

def calc_gini(y):
    m = len(y)
    if m == 0:
        return 0
    counts = Counter(y)
    probas = [c/m for c in counts.values()]
    impurity = 1 - sum([p**2 for p in probas])
    return impurity

# testing the function
y = [1, 1, 1, 1, 0, 0, 0]
print(calc_gini(y))

0.48979591836734704


In [8]:
def entropy(y):
    m = len(y)
    counts = Counter(y)
    probas = [c/m for c in counts.values()]
    return -sum([p * math.log(p, 2) for p in probas])

# testing the function
y = [1, 1, 1, 1, 0, 0, 0]
print(entropy(y))

0.9852281360342516
